In [1]:
# Generate Sinkhole Attributes

import numpy as np
import pandas as pd
import os
import datetime
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings("ignore")

from w210_attribute_library import haversine_distance, getdate, findname, findAttr, shAttributes, sh_attr2

In [2]:
# Directories
dir006 = "/Users/carlosmoreno/w210_images/jpegs006at64/"
datdir = "../data/"
attrs = "../attrs/"

In [3]:
# os.listdir("/Users/carlosmoreno/w210_images")

In [4]:
l006 = os.listdir(dir006)
eventID = []
for im in l006:
    eventID.append(int(im.split("-")[0]))
len(eventID) 

92

## Load Sinkhole Date for Florida (State)

In [5]:
df_sh = pd.read_excel(datdir+"Florida_Subsidence_Incident_Reports.xlsx", sheet_name=0)
df_sh["DateD"] = df_sh.apply(lambda row: getdate(row["EVENT_DATE"]), axis=1)
df_sh.head()

,X,Y,OBJECTID,REF_NUM,DATE_REV,EVENT_DATE,Year,TRUE_SINK,LONGDD,LATDD,...,CAVVIS,SUBRATE,PROPDAM,REPAIR_S,DRAINSTR,SOILTYPE,COMMENTS,COMMENTS_2,ACCESS_,DateD
0,-82.187100,27.758900,1,10-769,2015/12/23 00:00:00+00,1974/04/07 00:00:00+00,1974.0,U,-82.187097,27.758895,...,U,U,Y,Unknown,Unknown,UNKNOWN,BALM SINKHOLE. FIVE ORANGE TREES FELL INTO HOL...,"LEFT IN POSITION, AS GIVEN BY CK 10/5/12. ONE ...",9.1 MILES EAST OF S.R. 301 ON S.R. 672. 1350 F...,1974-04-07
1,-81.799854,28.667132,2,11-538,2016/04/28 00:00:00+00,1989/12/24 00:00:00+00,1989.0,U,-81.799850,28.667130,...,U,R,N,Planned,Unknown,UNK,THERE IS TWO SINKHOLES ON THIS SITE; SMALL SIN...,Accurate to the groves in section 10,"HI-ACRE GROVES, O'BRIEN RD. CLERMONT, FL ...",1989-12-24
2,-84.873189,30.712104,3,53-005,2021/07/29 00:00:00+00,NaN,NaN,U,-84.873185,30.712098,...,Y,U,N,Planned,Unknown,LIMESTONE,DROUGHT CONDITIONS. JUST OFF BANK IN LAKE SEMI...,Subsidence occured in June of 2000,. .,NaT
3,-84.322222,30.524722,4,55-009,2016/03/15 00:00:00+00,1999/09/16 00:00:00+00,1999.0,U,-84.322218,30.524717,...,Y,R,N,Not Repaired,Lake,ORGANIC SAND AND CLAY,MAY HAVE LS IN WALLS ASK TOM SCOTT. LAKE DRAIN...,NaN,AT END OF FAULK DRIVE. OFF US 27 NORTH.,1999-09-16
4,-82.620852,28.887506,5,02-504,2015/06/30 00:00:00+00,1983/03/23 00:00:00+00,1983.0,U,-82.620848,28.887501,...,Y,R,U,Unknown,Unknown,UNKNOWN,NEAR 520 515 538. ON LINE 522 536 501 507 505 ...,NaN,"HWY 44W. CRYSTAL RIVER, FL.",1983-03-23


## Load and Adjust Data for Sinkhole Events for Model (2019 to 2022 events) - Group = 0

In [6]:
dfevent = df_sh[(df_sh["OBJECTID"].isin(eventID))]
dfevent["Key"] = dfevent.apply(lambda row: str(row.OBJECTID)+"_1_0", axis=1)
dfevent["Sinkhole"] = 1
dfevent["name"] = dfevent.apply(lambda row: findname(row["OBJECTID"], l006), axis=1)
dfevent = dfevent[["Key", "name", "OBJECTID","Sinkhole" ,"X", "Y", "Year", "EVENT_DATE"]]
dfevent["DateD"] = dfevent.apply(lambda row: datetime.strptime(row["EVENT_DATE"][0:10],"%Y/%m/%d"), axis=1)
dfevent["Group"] = 0
dfevent.head()

,Key,name,OBJECTID,Sinkhole,X,Y,Year,EVENT_DATE,DateD,Group
427,428_1_0,428-2018-12-11-2019-02-09-6.0.jpeg,428,1,-81.932475,28.944928,2019.0,2019/12/11 00:00:00+00,2019-12-11,0
428,429_1_0,429-2019-01-05-2019-03-06-6.0.jpeg,429,1,-82.161323,29.208174,2020.0,2020/01/05 00:00:00+00,2020-01-05,0
430,431_1_0,431-2018-12-05-2019-02-03-6.0.jpeg,431,1,-82.625674,30.192630,2019.0,2019/12/05 00:00:00+00,2019-12-05,0
434,435_1_0,435-2019-06-01-2019-07-31-6.0.jpeg,435,1,-86.093300,30.682644,2020.0,2020/05/31 00:00:00+00,2020-05-31,0
543,544_1_0,544-2019-10-17-2019-12-16-6.0.jpeg,544,1,-81.323763,28.945974,2020.0,2020/10/16 00:00:00+00,2020-10-16,0


## Load and Adjust Data for Randomly Selected Y=0 Events (All over Florida) - Group = 1

In [7]:
revents = pd.read_csv(datdir+"final_satellite_random_FL.csv")
revents = revents[revents["label"]==0]
revents["Key"] = revents.apply(lambda row: str(row.ID)+"_0_1", axis=1)
revents["Sinkhole"] = 0
revents["Year"] = revents.apply(lambda row: list(df_sh[(df_sh["OBJECTID"] == row["ID"])]["Year"])[0], axis=1)
revents["EVENT_DATE"] = revents.apply(lambda row: list(df_sh[(df_sh["OBJECTID"] == row["ID"])]["EVENT_DATE"])[0], axis=1)
revents= revents[["Key", "name", "ID","Sinkhole" ,"lon", "lat", "Year", "EVENT_DATE"]]
revents.columns = ["Key", "name", "OBJECTID","Sinkhole" ,"X", "Y", "Year", "EVENT_DATE"]
revents["DateD"] = revents.apply(lambda row: datetime.strptime(row["EVENT_DATE"][0:10],"%Y/%m/%d"), axis=1)
revents["Group"] = 1
revents.head()

,Key,name,OBJECTID,Sinkhole,X,Y,Year,EVENT_DATE,DateD,Group
90,550_0_1,550-R-2020-06-20-2020-08-19-6.03-0.tif,550,0,-82.347574,24.985013,2021.0,2021/06/20 00:00:00+00,2021-06-20,1
91,2014_0_1,2014-R-2019-10-23-2019-12-22-6.03-0.tif,2014,0,-85.841454,30.982729,2020.0,2020/10/22 00:00:00+00,2020-10-22,1
92,2344_0_1,2344-R-2019-10-10-2019-12-09-6.03-0.tif,2344,0,-82.327277,29.309725,2020.0,2020/10/09 00:00:00+00,2020-10-09,1
93,2013_0_1,2013-R-2020-07-11-2020-09-09-6.03-0.tif,2013,0,-81.858988,28.115542,2021.0,2021/07/11 00:00:00+00,2021-07-11,1
94,1063_0_1,1063-R-2019-01-22-2019-03-23-6.03-0.tif,1063,0,-84.056783,30.253451,2020.0,2020/01/22 00:00:00+00,2020-01-22,1


## Load and Adjust Random Y=0 Events Closer to Sinkhole Event (Group = 2)

In [8]:
gr2 = pd.read_csv(datdir+"200at64predictions.csv")
gr2 = gr2[(gr2["label"] == 0)]
print(len(gr2))

92


In [9]:
gr2["Key"] = gr2.apply(lambda row: str(row.ID)+"_0_2", axis=1)
gr2["Sinkhole"] = 0
gr2["Year"] = gr2.apply(lambda row: list(df_sh[(df_sh["OBJECTID"] == row["ID"])]["Year"])[0], axis=1)
gr2["EVENT_DATE"] = gr2.apply(lambda row: list(df_sh[(df_sh["OBJECTID"] == row["ID"])]["EVENT_DATE"])[0], axis=1)
gr2= gr2[["Key", "name", "ID","Sinkhole" ,"lon", "lat", "Year", "EVENT_DATE"]]
gr2.columns = ["Key", "name", "OBJECTID","Sinkhole" ,"X", "Y", "Year", "EVENT_DATE"]
gr2["DateD"] = gr2.apply(lambda row: datetime.strptime(row["EVENT_DATE"][0:10],"%Y/%m/%d"), axis=1)
gr2["Group"] = 2
gr2.head()

,Key,name,OBJECTID,Sinkhole,X,Y,Year,EVENT_DATE,DateD,Group
88,2062_0_2,2062-2019-03-12-2019-05-11-200.00-0.jpeg_13_31...,2062,0,-82.185160,29.131709,2020.0,2020/03/11 00:00:00+00,2020-03-11,2
89,544_0_2,544-2019-10-17-2019-12-16-200.00-0.jpeg_13_31....,544,0,-81.346997,28.865996,2020.0,2020/10/16 00:00:00+00,2020-10-16,2
90,545_0_2,545-2020-07-23-2020-09-21-200.00-0.jpeg_13_31....,545,0,-81.393170,28.263308,2021.0,2021/07/23 00:00:00+00,2021-07-23,2
91,547_0_2,547-2020-08-07-2020-10-06-200.00-0.jpeg_13_31....,547,0,-81.911353,28.067834,2021.0,2021/08/07 00:00:00+00,2021-08-07,2
92,1060_0_2,1060-2018-12-18-2019-02-16-200.00-0.jpeg_13_31...,1060,0,-82.182879,29.131269,2019.0,2019/12/18 00:00:00+00,2019-12-18,2


## Merge Three DataFrames for Sinkhole Attributes Calculations

In [10]:
#Concat both Dataframes (with events and without)
finEvents = pd.concat([dfevent,revents, gr2])
len(finEvents)

276

In [11]:
finEvents.head()

,Key,name,OBJECTID,Sinkhole,X,Y,Year,EVENT_DATE,DateD,Group
427,428_1_0,428-2018-12-11-2019-02-09-6.0.jpeg,428,1,-81.932475,28.944928,2019.0,2019/12/11 00:00:00+00,2019-12-11,0
428,429_1_0,429-2019-01-05-2019-03-06-6.0.jpeg,429,1,-82.161323,29.208174,2020.0,2020/01/05 00:00:00+00,2020-01-05,0
430,431_1_0,431-2018-12-05-2019-02-03-6.0.jpeg,431,1,-82.625674,30.192630,2019.0,2019/12/05 00:00:00+00,2019-12-05,0
434,435_1_0,435-2019-06-01-2019-07-31-6.0.jpeg,435,1,-86.093300,30.682644,2020.0,2020/05/31 00:00:00+00,2020-05-31,0
543,544_1_0,544-2019-10-17-2019-12-16-6.0.jpeg,544,1,-81.323763,28.945974,2020.0,2020/10/16 00:00:00+00,2020-10-16,0


In [12]:
pivot = np.round(pd.pivot_table(finEvents, values='Year', 
                                index='Sinkhole', 
                                columns='Group', 
                                aggfunc='count'),2)
pivot

Group,0,1,2
Sinkhole,,,
0,NaN,92.0,92.0
1,92.0,NaN,NaN


## Generate Sinkhole Attributes (Using Tile Level Calculations)

In [13]:
daysdelta = timedelta(365)
fields = ["X", "Y", "OBJECTID"]
sh365 = shAttributes(df_sh, finEvents, daysdelta, fields)
sh365.to_excel(attrs+"model_sh_attr_365_1.xlsx", sheet_name="sh_attributes2")

26


In [14]:
daysdelta = timedelta(0)
fields = ["X", "Y", "OBJECTID"]
sh0 = shAttributes(df_sh, finEvents, daysdelta, fields)
sh0.to_excel(attrs+"model_sh_attr_0_1.xlsx", sheet_name="sh_attributes2")

26


## Generate Sinkhole Attributes - Using Joins

In [ ]:
df1 = pd.read_excel(datdir+"Florida_Subsidence_Incident_Reports.xlsx", sheet_name=0)

df_left = df1[["OBJECTID","EVENT_DATE","X", "Y"]]
df_left = df_left.dropna()
df_left["DateD"] = df_left.apply(lambda row: getdate(row["EVENT_DATE"]), axis=1)
print(df_left.count())
df_left.head(2)

In [ ]:
# dffinal = finEvents.merge(df_left, how="cross")
# print(len(dffinal)_
# dffinal.head(2)

In [ ]:
# dffinal["Distance_Miles"] = dffinal.apply(lambda row: haversine_distance(row["Y_x"], row["X_x"], row["Y_y"],row["X_y"], 3963.19), axis=1)

In [ ]:
# df2 = dffinal[dffinal["Distance_Miles"] !=0 ]
# len(df2)

In [ ]:
att_list = ["25","50", "75", "100", "150", "200",
            "250", "300", "500", "750", "1000"]

dist_list = [0.25,0.50, 0.75, 1.0, 1.5, 2.0, 2.50, 3.0, 5.0, 7.5, 10]

col_names = ["l25","l50", "l75", "l100", "l150", "l200",
             "l250", "l300", "l500", "l750", "l1000",
             "Y25", "Y50", "Y75", "Y100", "Y150", "Y200",
             "Y250", "Y300", "Y500", "Y750", "Y1000"]

In [ ]:
#Concat both Dataframes (with events and without)
finEvents1 = pd.concat([dfevent,revents, gr2])
len(finEvents1)

In [ ]:
ndays = 365
daysdelta = timedelta(ndays)
df_cols = list(finEvents1.columns)
datash = sh_attr2(finEvents1, df_left, daysdelta, att_list, dist_list, col_names)

In [ ]:
dfsh = pd.DataFrame(datash, columns = df_cols+col_names)
# print(dfsh.columns)
# dfsh.head(1)

In [ ]:
dfsh.to_excel(attrs+"model_sh_attr_365_2.xlsx", sheet_name="sh_attributes2")

In [ ]:
ndays = 0
daysdelta = timedelta(ndays)
df_cols = list(finEvents1.columns)
datash = sh_attr2(finEvents1, df_left, daysdelta, att_list, dist_list, col_names)

In [ ]:
dfsh = pd.DataFrame(datash, columns = df_cols+col_names)
dfsh.head(1)

dfsh.to_excel(attrs+"model_sh_attr_0_2.xlsx", sheet_name="sh_attributes2")